In [8]:
import sqlite3
import pandas as pd

# Connect to a temporary in-memory database (or use a file: 'mydb.sqlite')
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create and populate tables (copy your SQL script here)
cursor.executescript("""
PRAGMA foreign_keys = ON;

DROP TABLE IF EXISTS orders;
DROP TABLE IF EXISTS customers;

CREATE TABLE customers (
    id INTEGER PRIMARY KEY,
    name TEXT,
    country TEXT,
    email TEXT
);

CREATE TABLE orders (
    id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_date TEXT,
    total REAL,
    status TEXT,
    FOREIGN KEY (customer_id) REFERENCES customers(id)
);

INSERT INTO customers (id, name, country, email) VALUES
(1, 'Alice Smith', 'USA', 'alice@example.com'),
(2, 'Bob Johnson', 'Germany', 'bob@example.de'),
(3, 'Carlos Diaz', 'Spain', 'carlos@example.es'),
(4, 'Diana Lee', 'USA', 'diana@example.com'),
(5, 'Eva Müller', 'Germany', 'eva@gmail.com'),
(6, 'Franco Rossi', 'Italy', 'franco@example.it');

INSERT INTO orders (id, customer_id, order_date, total, status) VALUES
(101, 1, '2024-01-15', 250.00, 'shipped'),
(102, 1, '2024-02-10', 125.50, 'processing'),
(103, 2, '2024-02-15', 980.00, 'shipped'),
(104, 3, '2024-03-01', 1500.00, 'shipped'),
(105, 3, '2024-03-15', 80.00, 'canceled'),
(106, 4, '2024-04-10', 600.00, 'shipped'),
(107, 5, '2024-04-12', 240.00, 'shipped'),
(108, 5, '2024-04-13', 240.00, 'shipped'),
(109, 6, '2024-04-15', 75.00, 'processing');
""")


In [10]:
# Run a query
query = """
SELECT c.name, SUM(o.total) AS total_spent, AVG(o.total) AS avg_spent
FROM orders o
JOIN customers c ON o.customer_id = c.id
GROUP BY c.name;
"""

# Execute and display
df = pd.read_sql_query(query, conn)
df


,name,total_spent,avg_spent
0,Alice Smith,375.5,187.75
1,Bob Johnson,980.0,980.00
2,Carlos Diaz,1580.0,790.00
3,Diana Lee,600.0,600.00
4,Eva Müller,480.0,240.00
5,Franco Rossi,75.0,75.00


In [11]:
cursor.executescript("""
    CREATE TABLE newsletter_subscriptions (
    customer_id INTEGER PRIMARY KEY,
    subscribed INTEGER,  -- 1 = yes, 0 = no
    FOREIGN KEY (customer_id) REFERENCES customers(id)
);
""")

In [12]:
cursor.executescript("""
INSERT INTO 'newsletter_subscriptions' ('customer_id', 'subscribed') VALUES
  (1, 1),
  (3, 1),
  (6, 1);
""")

In [16]:
# Run a query
query = """
SELECT c.name, s.subscribed
FROM customers c
INNER JOIN newsletter_subscriptions s ON c.id = s.customer_id;
"""

# Execute and display
df = pd.read_sql_query(query, conn)
df


,name,subscribed
0,Alice Smith,1
1,Carlos Diaz,1
2,Franco Rossi,1


In [17]:
# Run a query
query = """
-- LEFT JOIN: all customers, even if not subscribed
SELECT c.name, s.subscribed
FROM customers c
LEFT JOIN newsletter_subscriptions s ON c.id = s.customer_id;
"""

# Execute and display
df = pd.read_sql_query(query, conn)
df

,name,subscribed
0,Alice Smith,1.0
1,Bob Johnson,NaN
2,Carlos Diaz,1.0
3,Diana Lee,NaN
4,Eva Müller,NaN
5,Franco Rossi,1.0


In [20]:
# Run a query
query = """
-- FULL JOIN alternative using UNION (SQLite doesnt' support FULL JOIN)
SELECT c.name, s.subscribed
FROM customers c
LEFT JOIN newsletter_subscriptions s ON c.id = s.customer_id

UNION

SELECT c.name, s.subscribed
FROM newsletter_subscriptions s
LEFT JOIN customers c ON c.id = s.customer_id;
"""

# Execute and display
df = pd.read_sql_query(query, conn)
df

,name,subscribed
0,Alice Smith,1.0
1,Bob Johnson,NaN
2,Carlos Diaz,1.0
3,Diana Lee,NaN
4,Eva Müller,NaN
5,Franco Rossi,1.0


In [26]:
# Run a query
query = """
-- Customers from USA or Germany, excluding those with Gmail
SELECT *
FROM customers
WHERE (country = 'USA' OR country = 'Germany')
  AND email NOT LIKE '%@gmail.com';
"""

# Execute and display
df = pd.read_sql_query(query, conn)
df

,id,name,country,email
0,1,Alice Smith,USA,alice@example.com
1,2,Bob Johnson,Germany,bob@example.de
2,4,Diana Lee,USA,diana@example.com


In [29]:
# Run a query
query = """
-- Total spending per customer, show only those over $500
SELECT c.name, SUM(o.total) AS total_spent
FROM customers c
JOIN orders o ON c.id = o.customer_id
GROUP BY c.name
HAVING total_spent > 500
;
"""

# Execute and display
df = pd.read_sql_query(query, conn)
df

,name,total_spent
0,Bob Johnson,980.0
1,Carlos Diaz,1580.0
2,Diana Lee,600.0


In [30]:
# Run a query
query = """
-- Subquery: get customers with above-average order totals
SELECT name
FROM customers
WHERE id IN (
    SELECT customer_id
    FROM orders
    GROUP BY customer_id
    HAVING AVG(total) > (
        SELECT AVG(total) FROM orders
    )
);
"""

# Execute and display
df = pd.read_sql_query(query, conn)
df

,name
0,Bob Johnson
1,Carlos Diaz
2,Diana Lee


In [31]:
# Run a query
query = """
-- CTE version
WITH avg_orders AS (
    SELECT customer_id, AVG(total) AS avg_total
    FROM orders
    GROUP BY customer_id
),
overall_avg AS (
    SELECT AVG(total) AS avg_all FROM orders
)
SELECT c.name, a.avg_total
FROM avg_orders a
JOIN customers c ON c.id = a.customer_id, overall_avg
WHERE a.avg_total > overall_avg.avg_all;
"""

# Execute and display
df = pd.read_sql_query(query, conn)
df

,name,avg_total
0,Bob Johnson,980.0
1,Carlos Diaz,790.0
2,Diana Lee,600.0


In [43]:
# Run a query
query = """
-- Rank orders by total per customer
SELECT
    o.id,
    c.name,
    o.total,
    RANK() OVER (PARTITION BY c.id ORDER BY o.total DESC) AS rank_within_customer
FROM orders o
JOIN customers c ON c.id = o.customer_id;
"""

# Execute and display
df = pd.read_sql_query(query, conn)
df

,id,name,total,rank_within_customer
0,101,Alice Smith,250.0,1
1,102,Alice Smith,125.5,2
2,103,Bob Johnson,980.0,1
3,104,Carlos Diaz,1500.0,1
4,105,Carlos Diaz,80.0,2
5,106,Diana Lee,600.0,1
6,107,Eva Müller,240.0,1
7,108,Eva Müller,240.0,1
8,109,Franco Rossi,75.0,1


In [48]:
# Run a query
query = """
select *, round(julianday('now') - julianday(order_date)) as days_from_order
from orders;
"""

# Execute and display
df = pd.read_sql_query(query, conn)
df


,id,customer_id,order_date,total,status,days_from_order
0,101,1,2024-01-15,250.0,shipped,456.0
1,102,1,2024-02-10,125.5,processing,430.0
2,103,2,2024-02-15,980.0,shipped,425.0
3,104,3,2024-03-01,1500.0,shipped,410.0
4,105,3,2024-03-15,80.0,canceled,396.0
5,106,4,2024-04-10,600.0,shipped,370.0
6,107,5,2024-04-12,240.0,shipped,368.0
7,108,5,2024-04-13,240.0,shipped,367.0
8,109,6,2024-04-15,75.0,processing,365.0
